In [1]:
import argparse
import logging
import os
import sys

import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, random_split

sys.path.append("..")
sys.path.append("../scripts/")
import superlayer.utils

from scripts import eval_net, train_net, get_args, train_block

from superlayer.models import SuperNet, SUnet
from superlayer.utils import BrainD, dice_coeff, one_hot, plot_img_array, plot_side_by_side

In [2]:
dir_img = '/home/gid-dalcaav/projects/neuron/data/t1_mix/proc/resize256-crop_x32-slice100/train/vols/'
dir_mask = '/home/gid-dalcaav/projects/neuron/data/t1_mix/proc/resize256-crop_x32-slice100/train/asegs/'

dir_train = '/home/vib9/src/SL-Net/jupyter/partitions/train.txt'
dir_val = '/home/vib9/src/SL-Net/jupyter/partitions/val.txt'

dir_checkpoint_1 = 'checkpoints_1/'
dir_checkpoint_2 = 'checkpoints_2/'

In [3]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
args = get_args()
device = torch.device('cuda')
logging.info(f'Using device {device}')

INFO: Using device cuda


In [4]:
target_label_numbers = [0,2,3,4,10,16,17,28,31,41,42,43,49,53,63]
val_percent = 0.1
batch_size = args.batchsize
lr = args.lr
img_scale = args.scale
writer = SummaryWriter(comment=f'LR_{lr}_BS_{batch_size}_SCALE_{img_scale}')

In [5]:
enc_nf = [128, 128, 128, 128]
dec_nf = [128, 128, 128, 128]
net1 = SUnet(input_ch=1, out_ch=15, use_bn=True, enc_nf=enc_nf, dec_nf=dec_nf, ignore_last=False)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net1.to(device=device)
train_scores1, val_scores1, train_var_1, val_var_1 = train_net(net=net1,
                                                                epochs=8,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

INFO: Network:
	1 input channels
	15 output channels (classes)

INFO: Creating dataset with 7329 examples
INFO: Creating dataset with 7329 examples
INFO: Starting training:
        Epochs:          8
        Batch size:      8
        Learning rate:   0.001
        Training size:   7329
        Validation size: 7329
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1
    
Validation round:  74%|███████▍  | 677/916 [00:42<00:11, 21.24batch/s]


Validation round: 100%|██████████| 916/916 [00:55<00:00, 22.17batch/s]
                                                                      INFO: Validation Dice Loss: 0.4411084273236287
Validation round:  53%|█████▎    | 485/916 [00:23<00:18, 22.69batch/s]


Validation round: 100%|██████████| 916/916 [00:43<00:00, 25.02batch/s]
                                                                      INFO: Validation Dice Loss: 0.45180288513544864
Validation round:  34%|███▎      | 308/916 [00:15<00:33, 18.39batch/s]


Validation round: 100%|█████████▉| 914/916 [00:45<00:00, 25.03batch/s]
                                                                      INFO: Validation Dice Loss: 0.44502994587327716
Validation round:  11%|█         | 97/916 [00:05<00:36, 22.46batch/s]


Validation round:  85%|████████▌ | 782/916 [00:38<00:06, 19.41batch/s]


...

Validation round: 100%|█████████▉| 914/916 [00:41<00:00, 25.03batch/s]
                                                                      INFO: Validation Dice Loss: 0.03751787599420183
Validation round:  82%|████████▏ | 752/916 [00:34<00:07, 22.99batch/s]


Validation round: 100%|██████████| 916/916 [00:41<00:00, 25.16batch/s]
                                                                      INFO: Validation Dice Loss: 0.0364210630104141
Validation round:  70%|██████▉   | 640/916 [00:29<00:15, 17.95batch/s]


Validation round: 100%|█████████▉| 914/916 [00:41<00:00, 23.88batch/s]
                                                                      INFO: Validation Dice Loss: 0.036312072794054795
Validation round:  58%|█████▊    | 531/916 [00:24<00:17, 21.78batch/s]


Validation round: 100%|█████████▉| 913/916 [00:41<00:00, 24.11batch/s]
                                                                      INFO: Validation Dice Loss: 0.03632821356593001
Validation round:  46%|████▌     | 423/916 [00:19<00:19, 24.69batch/s]


Validation round: 100%|█████████▉| 914/916 [00:41<00:00, 26.38batch/s]
                                                                      INFO: Validation Dice Loss: 0.035994785174922654
Epoch 8/8: 100%|██████████| 7329/7329 [04:47<00:00, 25.53img/s, loss (batch)=0.0615]
INFO: Checkpoint 8 saved !


In [6]:
np.save("/home/vib9/src/SL-Net/superlayer/models/finisblocks/first_128.npy",net1.block0.conv1.weight.cpu().data)
np.save("/home/vib9/src/SL-Net/superlayer/models/finisblocks/last_128.npy",net1.out_conv.weight.cpu().data)

In [5]:
fW = np.load("/home/vib9/src/SL-Net/superlayer/models/finisblocks/first_128.npy")
lW = np.load("/home/vib9/src/SL-Net/superlayer/models/finisblocks/last_128.npy")

In [7]:
net2 = SuperNet(input_ch=1, out_ch=15, use_bn=True, superblock_size=256, depth=4, W=None, firstW=fW, lastW=lW, learnB=True)

logging.info(f'Network:\n'
             f'\t{1} input channels\n'
             f'\t{15} output channels (classes)\n')

net2.to(device=device)
train_scores2, val_scores2, train_var_2, val_var_2 = train_block(net=net2,
                                                                epochs=8,
                                                                batch_size=args.batchsize,
                                                                lr=args.lr,
                                                                device=device,
                                                                img_scale=args.scale,
                                                                val_percent=args.val / 100,
                                                                checkpoint=2,
                                                                target_label_numbers=target_label_numbers,
                                                                writer=writer,
                                                                train_path=dir_train,
                                                                val_path=dir_val)

INFO: Network:
	1 input channels
	15 output channels (classes)

INFO: Creating dataset with 7329 examples
INFO: Creating dataset with 7329 examples
INFO: Starting training:
        Epochs:          8
        Batch size:      8
        Learning rate:   0.001
        Training size:   7329
        Validation size: 7329
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1
    
Epoch 1/8:   0%|          | 8/7329 [00:03<50:17,  2.43img/s, loss (batch)=0.924]

torch.Size([128, 256, 3, 3])
torch.Size([8, 1, 160, 192])
torch.Size([8, 128, 160, 192])


Epoch 1/8:   0%|          | 8/7329 [00:03<50:17,  2.43img/s, loss (batch)=0.92] 

torch.Size([128, 256, 3, 3])
torch.Size([8, 1, 160, 192])
torch.Size([8, 128, 160, 192])


Epoch 1/8:   0%|          | 8/7329 [00:04<1:15:51,  1.61img/s, loss (batch)=0.92]


RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.